In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
!pip install bert-tensorflow
from tensorflow import keras
import tensorflow_hub as hub
!pip install bert-for-tf2 # Just for tokenization, do not use it for model layer
from bert import bert_tokenization

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 40kB 1.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.6-cp36-none-any.whl size=30318 sha256=6da8b7bc3c262026c3374e1238ae3ab9550f811ce758bf862ffdcb6635dfb668
  Stored in directory: /root/.cache/pip/wheels/07/a0/b4/75b0601ebaa41e517a797fe9cea119c789664c8408f8a74ae9
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=74d2fc4bcb0b9391369078fcb47b095c9f2e01c5bd955c38fd08055cd5c83824
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=64393a03b55b9a583d5a0db6e804fa24f280596b1a6db41310853bd42d37944a
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
model_base = '/content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained'
model_name = "bert_multi_cased_L-12_H-768_A-12"
model_ckpt = os.path.join(model_base, model_name , "bert_model.ckpt")
max_seq_len = 128

In [ ]:
input_word_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
segment_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
input_mask = keras.layers.Input(shape=(max_seq_len,), dtype='int32')

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/{}/2".format(model_name), trainable=True)
pooled_output, _ = bert_layer([input_word_ids, input_mask, segment_ids])                         
output = keras.layers.Dense(1, activation='sigmoid', name='logit')(pooled_output)
model = keras.Model(inputs=[input_word_ids,input_mask,segment_ids], outputs=output)
model.build(input_shape=(None, max_seq_len))

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
import pandas as pd
labeled_data = pd.read_csv('/content/drive/My Drive/Address Matching/labled_data_until_iter1.csv', sep="|")

In [ ]:
labeled_data.drop(['Unnamed: 0'], axis = 1)

,postcode,address_left,address_right,label
0,06 830,"591 W.PUTNAM AVENUE,GREENWICH,CONNECTICUT,UNIT...","591 W.PUTNAM AVENUE,GREENWICH,CONNECTICUT",1
1,06 830,"591 W.PUTNAM AVENUE,GREENWICH,CONNECTICUT,UNIT...","591 PUTNAM AVENUE,CONNECTICUT",1
2,06 830,"591 W.PUTNAM AVENUE,GREENWICH,CONNECTICUT","591 PUTNAM AVENUE,CONNECTICUT",1
3,1,"HANOVER MILLS,MANCHESTER","LUDGATE HILL 39,LONDON,UNITED KINGDOM",0
4,10 003,"41 E 11TH STREET,NEW YORK,NEW YORK,UNITED STATES","40 WATER STREET,NEW YORK CITY,NEW YORK,UNITED ...",0
...,...,...,...,...
1464,AB12 4QZ,"67 ARGYLL PLACE,ABERDEEN","10 ARGYLL PLACE,ABERDEEN,UNITED KINGDOM",0
1465,AB12 4QZ,"51 ARGYLL PLACE,PORTLETHEN,SCOTLAND","10 ARGYLL PLACE,ABERDEEN,UNITED KINGDOM",0
1466,AB12 4TD,"15 FERN PLACE,ABERDEEN,UNITED KINGDOM","27 FERN PLACE,ABERDEEN,UNITED KINGDOM",0
1467,AB12 4TH,"8 HILLSIDE PLACE,ABERDEEN,UNITED KINGDOM","14 HILLSIDE PLACE,ABERDEEN,SCOTLAND",0


In [ ]:
address_left = labeled_data['address_left'].values + ','+ labeled_data['postcode'].values
address_right = labeled_data['address_right'].values + ','+ labeled_data['postcode'].values
label = labeled_data['label'].values

In [ ]:
iter1_false_negative_samples = pd.read_csv("/content/drive/My Drive/Address Matching/false_positive_samples_matching_addresses_from_mark_processed_and_shuffled_with_model_predicted_label.csv",sep="|")

In [ ]:
iter1_false_negative_samples[5000:]

,Unnamed: 0,l_address,r_address,orignal_noisy_label,model_predicted_prob,model_predicted_label
5000,84613,"No.16,Zhenxing East Road,Juxian County,Rizhao,...","莒县县城振兴东路16号,日照市, CN, 276500",1,0.173207,0
5001,68065,"经济技术开发区新竹路6号,日照市, CN, 276800","NO.6 XINZHU ROAD, ECONOMIC AND TECHNOLOGICAL D...",1,0.173229,0
5002,66199,"JAEN 2 1º,Sevilla, ES, 41001","TENIENTE CORONEL SEGUI 2 1ª,Sevilla, ES, 41001",1,0.173246,0
5003,153598,"7 Clifford Street,London, GB, W1S2FT","7 CLIFFORD STREET,LONDON, GB, W1S2FT",1,0.173285,0
5004,227476,"江干区市民街200号圣奥中央商务大厦2002室,杭州市, CN, 310000","ROOM 2391,2ND FLOOR,BUILDING 1,NO.104,WULIN NE...",1,0.173356,0
...,...,...,...,...,...,...
10395,69325,"市中区共青团路25号3904室,济南市, CN, 250012","Room 3904, 25 gongqingtuan road, shizhong dist...",1,0.499638,0
10396,58591,"2, Bld. 13, 3rd Yamskogo Polya Street, Office ...","улица Ямского Поля 3-я, дом 2, корп.13, офис X...",1,0.499675,0
10397,228376,"5470 South Tuckaway Lane,Greenfield, US, 53221","C/O GLENCO PROPERTIES INC, 5470 S TUCKAWAY LN,...",1,0.499690,0
10398,23357,"Gaoke Headoffice No. 1, No. 898 Zhuzhou Rd. Zh...","天元区株洲大道898号高科总部壹号,株洲市, CN, 412007",1,0.499837,0


In [ ]:
address_left = np.concatenate([address_left, iter1_false_negative_samples[5000:]['l_address']])
address_right = np.concatenate([address_right, iter1_false_negative_samples[5000:]['r_address']])
label = np.concatenate([label, [1] * len(iter1_false_negative_samples[5000:])])

In [ ]:
address_left = np.concatenate([address_left, iter1_false_negative_samples[5000:]['l_address']])
address_right = np.concatenate([address_right, iter1_false_negative_samples[4999: 4999 + len(iter1_false_negative_samples[5000:])]['r_address']])
label = np.concatenate([label, [0] * len(iter1_false_negative_samples[5000:])])

In [ ]:
len(address_left)

12269

In [ ]:
len(address_right)

12269

In [ ]:
shuffle_index = np.random.permutation(len(label))

In [ ]:
address_left = address_left[shuffle_index]
address_right = address_right[shuffle_index]
label = label[shuffle_index]

In [ ]:
import numpy

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

def process_data(address_left, address_right, labels, max_seq_len = 128):
  assert len(address_left) == len(address_right)
  assert len(address_right) == len(labels)
  input_id_list = []
  input_mask_list = []
  segment_id_list = []
  for example_index in range(len(address_left)):
      the_address_left_tokens = tokenizer.tokenize(address_left[example_index])
      the_address_right_tokens = tokenizer.tokenize(address_right[example_index])
      _truncate_seq_pair(the_address_left_tokens, the_address_right_tokens, max_seq_len - 3)
      tokens = ["[CLS]"] + the_address_left_tokens+ ["[SEP]"] + the_address_right_tokens + ["[CLS]"]
      segment_ids = ([0] * (len(the_address_left_tokens) + 2 )) + ([1] * (len(the_address_right_tokens) + 1 ))
      input_ids = tokenizer.convert_tokens_to_ids(tokens)
      input_mask = [1] * len(input_ids)
      padding = [0] * (max_seq_len - len(input_ids))
      input_ids += padding
      input_mask += padding
      segment_ids += padding
      assert len(input_ids) == max_seq_len
      assert len(input_mask) == max_seq_len
      assert len(segment_ids) == max_seq_len
      input_id_list.append(input_ids)
      input_mask_list.append(input_mask)
      segment_id_list.append(segment_ids)
  return np.array(input_id_list), np.array(input_mask_list), np.array(segment_id_list)

In [ ]:
input_ids, input_masks, segment_sids =  process_data(address_left, address_right, label)

In [ ]:
address_left

array(['6th Floor, 320 West 46th Street,New York, US, 10036',
       'Asia Square Tower 1, 8 Marina View, #30-01,Singapore, SG, 018960',
       '5-15-8, Ginza,Chuo-ku Tokyo, JP, 104-0061', ...,
       'Gongti Road Zhongyu Plaza,Chaoyang Qu, CN, 100027',
       '626 Water Street,Indiana, US, 15701',
       '高田三丁目23番23号,東京都 豊島区, JP, 171-0033'], dtype=object)

In [ ]:
address_right

array(['晴海三丁目5番1号,東京都 中央区, JP, 104-0053',
       '8 MARINA VIEW #30-01 ASIA SQUARE TOWER 1,SINGAPORE, SG, 018960',
       'C/O PATRICK T MCTIGUE, 31600 STEPHENSON HWY,MADISON HEIGHTS, US, 48071',
       ..., "O'HANLON HOUSE, LUTHER STREET,OXFORD, GB, OX1 1UL",
       '西新宿2丁目6番1号,東京都 新宿区, JP, 163-0220',
       '3-23-23,Takada,Toshima-ku Tokyo, JP, 171-0033'], dtype=object)

In [ ]:
input_ids_train = input_ids[:10000]
segment_id_train = segment_sids[:10000]
input_masks_train =  input_masks[:10000]

In [ ]:
input_ids_valid = input_ids[10000:]
segment_id_valid = segment_sids[10000:]
input_masks_valid =  input_masks[10000:]

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=1e-5, clipnorm=1.), 
              loss='binary_crossentropy', 
              metrics=['accuracy'], )
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 177853441   input_1[0][0]                    
                                                                 input_3[0][0]         

In [ ]:
num_train_epochs = 3
train_batch_size = 64

In [ ]:
from tensorflow.keras import callbacks
from sklearn.metrics import accuracy_score
import numpy as np

def polynomial_decay(global_step, lr):
  return (lr - 0.000001) * (1 - global_step / num_train_epochs) ** 0.9 + 0.000001

cb_lists = []

lr_scheduler = callbacks.LearningRateScheduler(polynomial_decay, verbose=1)
cb_lists.append(lr_scheduler)
 
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
      self.val_accuracy = []
      self.highest_accuracy = 0

    def on_batch_end(self, batch, logs={}):
      if batch % 10 == 0:
        predictions = self.model.predict([input_ids_valid, input_masks_valid, segment_id_valid])
        val_accuracy = accuracy_score(label[10000:], np.where(predictions>0.5, 1, 0))
        self.val_accuracy.append(val_accuracy)
        print(" - val_accuracy: {}".format(self.val_accuracy[-1]))
        if self.highest_accuracy < self.val_accuracy[-1]:
          self.highest_accuracy = self.val_accuracy[-1]
          print("Saving model to {} ...".format(os.path.join(model_base, model_name , "iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt")))
          self.model.save(
              os.path.join(model_base, model_name, "iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt")
          )


cb_lists.append(LossHistory())

In [ ]:
model.fit([input_ids_train, input_masks_train, segment_id_train], label[:10000], 
          validation_data=([input_ids_valid, input_masks_valid, segment_id_valid], label[10000:]), 
          epochs = num_train_epochs, callbacks = cb_lists, shuffle = True, batch_size=32) 


Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-06.
Epoch 1/50
 - val_accuracy: 0.8210665491405905
Saving model to /content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained/bert_multi_cased_L-12_H-768_A-12/iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained/bert_multi_cased_L-12_H-768_A-12/iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained/bert_multi_cased_L-12_H-768_A-12/iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt/assets


230/313 [=====================>........] - ETA: 7:33 - loss: 0.0195 - accuracy: 0.9958 - val_accuracy: 0.9938298810048479
Saving model to /content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained/bert_multi_cased_L-12_H-768_A-12/iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt ...
INFO:tensorflow:Assets written to: /content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained/bert_multi_cased_L-12_H-768_A-12/iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Address Matching/iter2_bert_model_mixed_golden_and_false_positive_from_iter1_trained/bert_multi_cased_L-12_H-768_A-12/iter2_bert_model_multi_lang_address_gold_and_fp_values_from_iter1_fineturned.ckpt/assets


313/313 [==============================] - 1818s 6s/step - loss: 0.0190 - accuracy: 0.9960 - val_loss: 0.0276 - val_accuracy: 0.9925

Epoch 00004: LearningRateScheduler reducing learning rate to 1e-06.
Epoch 4/50
 - val_accuracy: 0.9925077126487439
313/313 [==============================] - 1675s 5s/step - loss: 0.0142 - accuracy: 0.9967 - val_loss: 0.0277 - val_accuracy: 0.9934


ValueError: ignored